In [ ]:
import h5py
import pandas as pd 
import scanpy as sc
import anndata

In [ ]:
import wot
import matplotlib.pyplot as plt

In [ ]:
adata=sc.read_loom("/For_optimal_transport/Genescore_foropt_fetalheart1.loom")#reading in the loom object of gene score

In [ ]:
adata_cell_details=pd.read_csv('For_optimal_transport/Cell_details.csv')#loading the barcode details
adata_cell_details.index=adata_cell_details['index']
print(adata_cell_details[0:5])


In [ ]:
adata.obs['Clusters1']=adata_cell_details['Clusters1']
adata.obs['index']=adata_cell_details['index']

In [ ]:
adata.obs['Day']=adata.obs['orig_ident']
adata.obs['Day']=adata.obs['Day'].str.replace('F19','3')
adata.obs['Day']=adata.obs['Day'].str.replace('F8','2')
adata.obs['Day']=adata.obs['Day'].str.replace('F6','1')
adata.obs['Day']=adata.obs['Day'].astype(int)
adata.obs[0:5]

In [ ]:
figure = plt.figure(figsize=(10, 10))
plt.axis('off')
plt.tight_layout()
plt.scatter(pd.DataFrame(adata.obsm['UMAP_cell_embeddings'])[0], pd.DataFrame(adata.obsm['UMAP_cell_embeddings'])[1],c=adata.obs['cell_growth_rate'],
               s=4, marker=',', edgecolors='none', alpha=0.8)
cb = plt.colorbar()
#cb.ax.set_title('Day')


In [ ]:
figure = plt.figure(figsize=(10, 10))
plt.axis('off')
plt.tight_layout()
plt.scatter(pd.DataFrame(adata.obsm['UMAP_cell_embeddings'])[0], pd.DataFrame(adata.obsm['UMAP_cell_embeddings'])[1],c=adata.obs['Day'],
               s=4, marker=',', edgecolors='none', alpha=0.8)
cb = plt.colorbar()
cb.ax.set_title('Day')


In [ ]:
GENE_SETS_PATH='For_optimal_transport/cellcycle_apoptosis.gmx'#using the gene sets to get the cell cycle and apoptosis signal

In [ ]:
GENE_SET_SCORES_PATH = 'For_optimal_transport/cellcyle_apoptosis_gene_set_scores.csv'#path for the file write

In [ ]:
gs = wot.io.read_sets(GENE_SETS_PATH, adata.var.index.values)

In [ ]:
import ipywidgets as widgets
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import wot


In [ ]:
gene_set_scores_df = pd.DataFrame(index=adata.obs.index)
print(gene_set_scores_df[0:5])

In [ ]:
for j in range(gs.shape[1]):
    gene_set_name = str(gs.var.index.values[j])
    result = wot.score_gene_sets(ds=adata, gs=gs[:, [j]], permutations=0, method='mean_z_score')
    gene_set_scores_df[gene_set_name] = result['score']
gene_set_scores_df.to_csv(GENE_SET_SCORES_PATH, index_label='id')

In [ ]:
gene_set_scores_df = pd.read_csv(GENE_SET_SCORES_PATH,index_col='id')
gene_set_dropdown = widgets.Dropdown(
    options=gene_set_scores_df.columns,
    description='Gene Set:'
)



In [ ]:
coord_df=pd.DataFrame(adata.obsm['UMAP_cell_embeddings'])
coord_df.columns=['x','y']
coord_df.index=adata.obs.index
print(coord_df[0:5])


In [ ]:
coord_df.to_csv('For_optimal_transport/coords_df.csv')#storing the umap coordinates

In [ ]:
days_df=pd.DataFrame(adata.obs['Day'])

In [ ]:
gene_set_scores_df = gene_set_scores_df.join(coord_df).join(days_df)
print(gene_set_scores_df[0:5])

In [ ]:
unique_days = adata.obs['Day'].unique()

In [ ]:
day_selector = widgets.SelectionRangeSlider(
    options=set(unique_days),
    continuous_update=False,
    index=(0,2),
    description='Weeks'
)

In [ ]:
day_selector

In [ ]:
import matplotlib 
matplotlib.rcParams['pdf.fonttype']=42
from matplotlib import pyplot as plt

def update_gene_set_vis(name, days):
    gene_set_score_coords = gene_set_scores_df[(gene_set_scores_df['Day']>=days[0]) & (gene_set_scores_df['Day']<=days[1])]
    figure = plt.figure(figsize=(7, 7))
    #plt.axis('off')
    plt.tight_layout()
    plt.title(name + ', Weeks {}-{}'.format(days[0], days[1]))
    plt.scatter(coord_df['x'], coord_df['y'], c='#d0d0d0',
                   s=4, marker=',', edgecolors='none', alpha=0.8)
    plt.scatter(gene_set_score_coords['x'], gene_set_score_coords['y'], c=gene_set_score_coords[name],
                   s=4, marker=',', edgecolors='none')
    cb = plt.colorbar()
    cb.ax.set_title('Signature')
    plt.savefig('OT_trajectory_apoptosis.pdf',dpi=1200)
    figure2 = plt.figure(figsize=(10, 5))
    plt.title(name + ', Weeks {}-{}'.format(days[0], days[1]))
    plt.hist(gene_set_score_coords[name])
    
    return figure, figure2

widgets.interact(update_gene_set_vis, name=gene_set_dropdown, days=day_selector)

In [ ]:
cell_sets=adata.obs.groupby('Clusters1')['index'].apply(list).to_dict()


In [ ]:
fout = "For_optimal_transport/major_cell_set.gmt"#A dictionary of barcode to cluster relationship
fo = open(fout, "w")

for k, v in cell_sets.items():
    fo.write(str(k) + '\t-\t'+ str(v) + '\n')

fo.close()
#replace all these characters [', in the file

In [ ]:
cell_sets_path='For_optimal_transport/major_cell_set.gmt'#loading the barcode to cluster dictionary

In [ ]:
cell_sets = wot.io.read_sets(cell_sets_path)

In [ ]:
import ipywidgets as widgets
# Visualize Cell Sets 
cell_set_dropdown = widgets.Dropdown(
    options=cell_sets.var.index,
    description='Cell Set:'
)


In [ ]:

day_selector = widgets.SelectionRangeSlider(
    options=unique_days,
    continuous_update=False,
    index=(0,len(unique_days)-1),
    description='Weeks'
)

In [ ]:

def update_cell_set_vis(name, days):
    cell_set = cell_sets[:, name]
    cell_set_coords = cell_set[cell_set.X>0].obs.join(coord_df).join(days_df)
    cell_set_coords = cell_set_coords[(cell_set_coords['Day']>=days[0]) & (cell_set_coords['Day']<=days[1])]
    figure = plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.tight_layout()
    plt.title(name + ', Weeks {}-{}, {:,} cells'.format(days[0], days[1], cell_set_coords.shape[0]))
    plt.scatter(coord_df['x'], coord_df['y'], c='#f0f0f0',
                   s=4, marker=',', edgecolors='none', alpha=0.8)
    plt.scatter(cell_set_coords['x'], cell_set_coords['y'], c=cell_set_coords['Day'],
                   s=4, marker=',', edgecolors='none', vmin=unique_days[0],  vmax=unique_days[len(unique_days)-1])
    cb = plt.colorbar()
    cb.ax.set_title('Week')

widgets.interact(update_cell_set_vis, name=cell_set_dropdown, days=day_selector)


In [ ]:
#making a copy of gene_set_scores_df
gene_set_scores=gene_set_scores_df

In [ ]:
proliferation=gene_set_scores['Cell.cycle']
apoptosis = gene_set_scores['Apoptosis']

print(gene_set_scores_df[0:5])
print(gene_set_scores[0:5])
print(proliferation[0:5])
print(apoptosis[0:5])

In [ ]:
# apply logistic function to transform to birth rate and death rate
def logistic(x, L, k, x0=0):
    f = L / (1 + np.exp(-k * (x - x0)))
    return f
def gen_logistic(p, beta_max, beta_min, pmax, pmin, center, width):
    return beta_min + logistic(p, L=beta_max - beta_min, k=4 / width, x0=center)

def beta(p, beta_max=1.7, beta_min=0.3, pmax=1.0, pmin=-0.5, center=0.25):
    return gen_logistic(p, beta_max, beta_min, pmax, pmin, center, width=0.5)

def delta(a, delta_max=1.7, delta_min=0.3, amax=0.5, amin=-0.4, center=0.1):
    return gen_logistic(a, delta_max, delta_min, amax, amin, center,
                          width=0.2)

birth = beta(proliferation)
death = delta(apoptosis)

print(birth[0:5])
print(death[0:5])

gr = np.exp(birth-death)
growth_rates_df = pd.DataFrame(index=gene_set_scores.index, data={'cell_growth_rate':gr})
growth_rates_df.to_csv('For_optimal_transport/growth_gs_init.txt')#write the growth rate of barcodes 
#This file will be used by the OT algorithm when run with peaks
